In [1]:
# od pair
from env import env
import pandas as pd
import random as rd

# here requests
from haversine import haversine, Unit
import flexpolyline as fp
import urllib.request
import json, sys

# map plot
from bs4 import BeautifulSoup as bs
import gmplot

### Get od pair

In [17]:
# try later: https://geobgu.xyz/py/shapely.html
"""
df = pd.read_csv('results/bike-results/result.csv')

lat = df['station_lat'].to_list()
lng = df['station_lng'].to_list()

rd.seed(31)
x = rd.sample(range(0,2191),2)

originCoord = (lat[x[0]],lng[x[0]])
origin = f'{originCoord[0]},{originCoord[1]}'

destinationCoord = (lat[x[1]],lng[x[1]])
destination = f'{destinationCoord[0]},{destinationCoord[1]}'

print(origin)
print(originCoord)
print(destination)
print(destinationCoord)
"""


41.9106,-87.6494
(41.9106, -87.6494)
41.76,-87.58
(41.76, -87.58)


In [12]:
filtered = pd.read_csv('datasets/new_bike/filtered_dataset_gas.csv')
odDf = pd.read_csv('datasets/new_bike/set_id_coord.csv')

In [3]:
# coletar par od qualquer para teste

ind = 0 # filtered route index
od = odDf.loc[odDf['setID'] == filtered.iloc[ind]['setID']]

origin = od['departure_coord'].to_string(index=False).split(":")
origin = f"{origin[1]},{origin[0]}"

destination = od['arrival_coord'].to_string(index=False).split(":")
destination = f"{destination[1]},{destination[0]}"

float_origin = list(map(lambda x: float(x),origin.split(',')))
float_destination = list(map(lambda x: float(x),destination.split(',')))

print(origin)
print(destination)
print(float_origin)
print(float_destination)

51.533778,-0.2677
51.57419,-0.46799
[51.533778, -0.2677]
[51.57419, -0.46799]


### Here requests

Car co2 and fuel consumption link: https://developer.here.com/documentation/routing-api/dev_guide/topics/use-cases/calculate-fuel-consumption.html

In [25]:
def route_midpoint(route_length,zipList):#,latList,lngList):
    middle = delta = route_length/2 # route length was hereResponse['routes'][0]['sections'][0]['summary']['length']/2
    #zipList = list(zip(latList,lngList))
    before = after = None
    dist = 0

    for i in range(len(zipList)-1):
        dist += haversine(zipList[i],zipList[i+1], unit=Unit.METERS)
        sub = abs(middle-dist)
        if sub < delta:
            delta = sub
            before = zipList[i+1]
        if sub > delta:
            after = zipList[i+1]
            break

    midpointCoord = ((before[0]+after[0])/2,(before[1]+after[1])/2)
    midpoint = f'{midpointCoord[0]},{midpointCoord[1]}'

    return midpoint,midpointCoord

# get closest bike share point on coordinate
def closest(coord: list): #### need refactor!!
    data = pd.read_csv('results/bike-results/result.csv')
    dist = sys.float_info.max
    waypoint = ()
    for ind,row in data.iterrows():
        distTest = haversine(coord,(row['station_lat'],row['station_lng']))
        if distTest < dist:
            dist = distTest
            waypoint = (row['station_lat'],row['station_lng'])
    return waypoint


def closest_list(coord: list, data: list):
    dist = sys.float_info.max
    waypoint = ()
    for i in data:
        distTest = haversine(coord,(float(i[0]),float(i[1])))
        if distTest < dist:
            dist = distTest
            waypoint = (float(i[0]),float(i[1]))
    return waypoint

In [6]:
def car_routing(strCoord,maxAlternatives=0):
    query = f'destination={strCoord[1]}&origin={strCoord[0]}'
    hereKey = env('HERE_API_KEY')
    
    fuelConsumption = '&fuel[type]=diesel&fuel[freeflowspeedtable]=0,0,60,0.1,80,0.12&fuel[trafficspeedtable]=0,0,60,0.1,80,0.12&fuel[additionalconsumption]=1.8&fuel[ascent]=0.1'

    url = 'https://router.hereapi.com/v8/routes?transportMode=%s%s&%s&alternatives=%d&return=summary,polyline&apikey=%s' % ('car',fuelConsumption,query,maxAlternatives,hereKey)
    response = urllib.request.urlopen(url).read().decode('utf-8')
    return [json.loads(response)]


def transit_routing(strCoord,maxAlternatives=0):
    query = f'destination={strCoord[1]}&origin={strCoord[0]}'
    hereKey = env('HERE_API_KEY')

    # change fuel consumption for transport if possible
    #fuelConsumption = '&fuel[type]=diesel&fuel[freeflowspeedtable]=0,0,60,0.1,80,0.12&fuel[trafficspeedtable]=0,0,60,0.1,80,0.12&fuel[additionalconsumption]=1.8&fuel[ascent]=0.1'

    # multimodal here url
    # transposrt types:
    # vehicle
    # transit
    # taxi
    # rented
    
    url = 'https://intermodal.router.hereapi.com/v8/routes?%s&transit[enable]=entireRoute&rented[enable]=&taxi[enable]=&vehicle[enable]=&alternatives=%d&return=travelSummary,polyline&apiKey=%s' % (query,maxAlternatives,hereKey)
    response = urllib.request.urlopen(url).read().decode('utf-8')
    return [json.loads(response)]


# for pedestrian: https://developer.here.com/documentation/routing-api/dev_guide/topics/pedestrian-defined.html
def bicycle_routing(strCoord,floatCoord,maxAlternatives=0):
    hereKey = env('HERE_API_KEY')

    newOrigin = closest(floatCoord[0]) # data
    newDestination = closest(floatCoord[1]) # data

    mode = ['pedestrian','bicycle','pedestrian']

    query = [
        f'destination={newOrigin[0]},{newOrigin[1]}&origin={strCoord[0]}',
        f'destination={newDestination[0]},{newDestination[1]}&origin={newOrigin[0]},{newOrigin[1]}',
        f'destination={strCoord[1]}&origin={newDestination[0]},{newDestination[1]}'
    ]

    responses = []

    for i in range(len(mode)):
        if mode[i] == 'pedestrian': altNum = 0
        else: altNum = maxAlternatives
        url = 'https://router.hereapi.com/v8/routes?transportMode=%s&%s&alternatives=%d&return=summary,polyline&apikey=%s' % (mode[i],query[i],altNum,hereKey)
        response = urllib.request.urlopen(url).read().decode('utf-8')
        responses.append(json.loads(response))

    return responses,newDestination

In [7]:
def response_reader(response,midpoint=False): # retorna polilinha das respostas
    polyline = []
    distance = 0
    duration = 0

    for section in response['routes'][0]['sections']:
        try:
            distance += section['summary']['length']
            duration += section['summary']['duration']
        except:
            distance += section['travelSummary']['length']
            duration += section['travelSummary']['duration']
        polyline += fp.decode(section['polyline'])

    if midpoint: return duration,distance,polyline,route_midpoint(distance,polyline)
    return duration,distance,polyline

In [17]:
coords = (origin,destination)
float_coords = (float_origin,float_destination)

response_full_car = car_routing(coords) # full car
response_full_transit = transit_routing(coords) # full transit
#responses_full_bicycle,_ = bicycle_routing(coords,float_coords) # full bicycle

In [18]:
duration_full_car,distance_full_car,polyline_full_car,car_midpoint = response_reader(*response_full_car,True)
duration_full_transit,distance_full_transit,polyline_full_transit,transit_midpoint = response_reader(*response_full_transit,True)
'''
duration_full_bicycle = 0
distance_full_bicycle = 0
polyline_full_bicycle = []

for response in responses_bicycle:
    duration, distance, polyline = response_reader(response)
    duration_full_bicycle += duration
    distance_full_bicycle += distance
    polyline_full_bicycle += polyline

bicycle_midpoint = route_midpoint(distance_full_bicycle,polyline_full_bicycle)
'''

In [19]:
response_car = car_routing((origin,car_midpoint[0]))
response_car_transit = transit_routing((car_midpoint[0],destination))
#response_car_bicycle,_ = bicycle_routing((car_midpoint[0],destination))

response_transit = transit_routing((origin,transit_midpoint[0]))
response_transit_car = car_routing((transit_midpoint[0],destination))
#responses_transit_bicycle,_ = bicycle_routing((transit_midpoint[0],destination))

#responses_bicycle,new_bicycle_midpoint = bicycle_routing((origin,bicycle_midpoint[0]))
#response_bicycle_car = car_routing((new_bicycle_midpoint,destination))
#response_bicycle_transit = transit_routing((new_bicycle_midpoint,destination))

In [20]:
duration_car,distance_car,polyline_car = response_reader(*response_car)
duration_car_transit,distance_car_transit,polyline_car_transit = response_reader(*response_car_transit)
#duration_car_bicycle,distance_car_bicycle,polyline_car_bicycle = response_reader(response_car_bicycle)

duration_transit,distance_transit,polyline_transit = response_reader(*response_transit)
duration_transit_car,distance_transit_car,polyline_transit_car = response_reader(*response_transit_car)
#duration_transit_bicycle,distance_transit_bicycle,polyline_transit_bicycle = response_reader(responses_transit_bicycle)

#duration_bicycle,distance_bicycle,polyline_bicycle = response_reader(responses_bicycle)
#duration_bicycle_car,distance_bicycle_car,polyline_bicycle_car = response_reader(response_bicycle_car)
#duration_bicycle_transit,distance_bicycle_transit,polyline_bicycle_transit = response_reader(response_bicycle_transit)

print(duration_car+duration_car_transit,distance_car+distance_car_transit) # car transit data
#print(duration_car+duration_car_bicycle,distance_car+distance_car_bicycle) # car bicycle data
print(duration_transit+duration_transit_car,distance_transit+distance_transit_car) # transit car data
#print(duration_transit+duration_transit_bicycle,distance_transit+distance_transit_bicycle) # transit bicycle data
#print(duration_bicycle+duration_bicycle_car,distance_bicycle+distance_bicycle_car) # bicycle car data
#print(duration_bicycle+duration_bicycle_transit,distance_bicycle+distance_bicycle_transit) # bicycle transit data

4493 19168
4176 24403


In [42]:
def polylineToCoord(tracer):
    tracer = tracer.replace(':',',').split(';')
    coordinates = []
    for i in tracer:
        coord = i.split(',')
        coordinates.append(((float(coord[0])),(float(coord[1]))))
    return coordinates

In [45]:
def map_plot(origin,og_polyline,new_polyline_list,color_list,fileName):
    origin_coord = origin.split(',')
    outputMap = gmplot.GoogleMapPlotter(float(origin_coord[0]),float(origin_coord[1]), 12)
    mapUrl = f'results/map_plot/{fileName}.html'

    lats = [t[1] for t in og_polyline]
    lngs = [t[0] for t in og_polyline]
    
    outputMap.plot(lats,lngs, color="black", edge_width=3)

    for i in range(len(new_polyline_list)):

        new_lats = [t[0] for t in new_polyline_list[i]]
        new_lngs = [t[1] for t in new_polyline_list[i]]

        outputMap.plot(new_lats, new_lngs, color=color_list[i], edge_width=3)

    outputMap.draw(mapUrl)

    page = open(mapUrl)
    soup = bs(page.read(),"html.parser")
    srcKey  = f"https://maps.googleapis.com/maps/api/js?key={env('GOOGLE_MAPS_KEY')}&libraries=visualization&sensor=true_or_false"
    line = soup.find('script',type="text/javascript")
    line['src'] = srcKey

    with open(mapUrl, "wb") as f_output:
        f_output.write(soup.prettify("utf-8")) 

In [46]:
treated_polyline = polylineToCoord(filtered.iloc[ind]['coordinates'])

map_plot(origin,treated_polyline,[polyline_full_car],["blue"],"full_car")
map_plot(origin,treated_polyline,[polyline_full_transit],["red"],"full_transit")
#map_plot(origin,treated_polyline,[polyline_full_bicycle],["green"],"full_bicycle")

map_plot(origin,treated_polyline,[polyline_car,polyline_car_transit],['blue','red'],'car_transit')
map_plot(origin,treated_polyline,[polyline_transit,polyline_transit_car],['red','blue'],'transit_car')
#map_plot(origin,treated_polyline,[polyline_transit,polyline_transit_bicycle],['red','green'],'transit_bicycle')


In [88]:
 # transit value test

data = filtered['midpoint'].to_list()
data = list(map(lambda x: x.split(","),data))
hope = closest_list(transit_midpoint[1],data)

dataf_hope = filtered[filtered['midpoint'] == f'{hope[0]},{hope[1]}']['coordinates'].values
treated_hope = polylineToCoord(*dataf_hope)

map_plot(origin,treated_hope,[polyline_full_transit],["red"],"transit_value_test")

[(-0.2677, 51.533778), (-0.2677, 51.53326), (-0.26763, 51.53323), (-0.26758, 51.53319), (-0.26758, 51.53314), (-0.26656, 51.53315), (-0.26555, 51.53321), (-0.26507, 51.5332), (-0.26479, 51.53319), (-0.26477, 51.53324), (-0.26461, 51.53337), (-0.26426, 51.53347), (-0.26347, 51.53367), (-0.26295, 51.53381), (-0.26234, 51.534), (-0.26189, 51.53411), (-0.2617, 51.53415), (-0.26148, 51.53418), (-0.26128, 51.53418), (-0.26111, 51.53416), (-0.26094, 51.53412), (-0.26076, 51.53407), (-0.2607, 51.53401), (-0.26064, 51.53393), (-0.26059, 51.53385), (-0.26054, 51.53375), (-0.26033, 51.53321), (-0.26025, 51.53302), (-0.2602, 51.53284), (-0.26018, 51.53268), (-0.26015, 51.532), (-0.26015, 51.53198), (-0.2601, 51.5316), (-0.26005, 51.53123), (-0.25997, 51.53062), (-0.25997, 51.53053), (-0.25988, 51.53013), (-0.25977, 51.52986), (-0.2596, 51.52946), (-0.25939, 51.52899), (-0.25933, 51.52891), (-0.25928, 51.52892), (-0.25923, 51.52891), (-0.25919, 51.52889), (-0.25918, 51.52886), (-0.25919, 51.52883),